# Login Authentication Mock Data

Notebook de desenvolvimento responável por realizar processo de mock dos dados e evenviar os dados para o eventhub utilizando autenticação com SPN


<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://raw.githubusercontent.com/Foiac/MobileFraudDetectSolution/main/Editaveis/Imagens/login-authetication-mock-data.png" alt="SparkStreaming Ingest" style="width: 800px">
</div>

#### Import dependecies

In [ ]:
import random
import json
from faker import Faker
from datetime import datetime
from azure.eventhub import EventHubProducerClient, EventData
from azure.identity import ClientSecretCredential
import pandas as pd
import threading

#### Path definitions and secrets

In [ ]:
azure_tenant_id: str = "dc25df03-ffa5-4111-b188-46fe6cd26a3a"
azure_client_id: str = ""
azure_client_secret: str = "" 
eventhub_namespace: str = "fraud-eh-namespace"
eventhub_namespace_fully: str = f"{eventhub_namespace}.servicebus.windows.net"
eventhub_name: str = "fraud-detect"

In [ ]:
file_path = 'phonedataset.csv'

df = pd.read_csv(file_path, sep=';')
phone_list = df.values.tolist()

with open('city_clusters.json', 'r', encoding='utf-8') as file:
    city_clusters = json.load(file)

#### Initial Definitions

In [ ]:
start_date = datetime(2024, 11, 1, 0)
end_date = datetime(2024, 11, 1, 23)

In [ ]:
fake = Faker()

transaction_prob = [0.78, 0.22] 
app_version_prob = [0.05, 0.20, 0.75]  
error_prob = [0.90, 0.08, 0.02] 
uids = [str(fake.random_number(digits=11, fix_len=True)) for _ in range(1000)]

#### Function to Create SPN Client 

In [9]:
def spn_authentication():
    
    print("init procces")
    credential = ClientSecretCredential(tenant_id=azure_tenant_id, 
                                        client_id=azure_client_id,
                                        client_secret=azure_client_secret)
    print("credential create")
    producer = EventHubProducerClient(fully_qualified_namespace=eventhub_namespace_fully,
                                        eventhub_name=eventhub_name,
                                        credential=credential)
    print("producer create")

    return producer

#### Function to Generate Random Data

In [ ]:
def generate_random_data():
    """Gera um dicionário com dados aleatórios no formato JSON especificado."""

    city = random.choice(city_clusters)
    latitude = random.gauss(city["lat"], city["std_dev"])
    longitude = random.gauss(city["long"], city["std_dev"])

    brand_version = random.choice(phone_list)
    phone_brand = brand_version[0]
    operational_version = brand_version[1]

    uid = random.choice(uids) if random.random() < 0.01 else str(fake.random_number(digits=11, fix_len=True))

    return {
        "imei": str(fake.random_number(digits=15, fix_len=True)),
        "mac": ':'.join([f"{random.randint(0, 255):02X}" for _ in range(6)]),
        "network": random.choice(["VIVO", "TIM", "CLARO", "OI"]),
        "client_ip": fake.ipv4(),
        "latitude": latitude,
        "longitude": longitude,
        "uid": uid,
        "password": fake.password(length=6, special_chars=False, digits=True, upper_case=True, lower_case=True),
        "transaction": random.choices(["true", "false"], weights=transaction_prob, k=1)[0],
        "api": "login-authentication",
        "endpoint": "v1/login",
        "os": operational_version,
        "phone_brand": phone_brand,
        "app_version": random.choices(["2.0.1", "2.0.2", "2.0.3"], weights=app_version_prob, k=1)[0],
        "error": random.choices(["0", "INCORRECT_PASS", "USER_NOT_FOUND"], weights=error_prob, k=1)[0],
        "timestamp": str(fake.unix_time(start_datetime=start_date, end_datetime=end_date) * 1000)
    }

#### Function to Send Data do Eventhub

In [12]:
def send_message_to_eventhub(eh_producer, msg_number, trh_number):
    """Abre client com eventhub e envia batch de mensagens."""
    for i in range(msg_number):

        message = json.dumps(generate_random_data())

        try:
            with eh_producer:
                event_data_batch = eh_producer.create_batch()

                event_data_batch.add(EventData(message))

                eh_producer.send_batch(event_data_batch)
                
                print(f"[{trh_number}] - Mensagem enviada com sucesso! - {message}")
        except ValueError as ve:
            print(f"Erro ao enviar mensagem: {ve}")
        except Exception as e:
            print(f"Erro inesperado ao enviar mensagem: {e}")
        finally:
            eh_producer.close()


#### Start Thread to Send Data

In [48]:
eh_producer = spn_authentication()

msg_number = 200

thread_1 = threading.Thread(target=send_message_to_eventhub, args = (eh_producer, msg_number, 1))
thread_2 = threading.Thread(target=send_message_to_eventhub, args = (eh_producer, msg_number, 2))

thread_1.start()
thread_2.start()

thread_1.join()
thread_2.join()

init procces
credential create
producer create
[1] - Mensagem enviada com sucesso! - {"imei": "672967386917408", "mac": "F8:CD:61:17:D7:66", "network": "TIM", "client_ip": "75.47.203.206", "latitude": -22.87215195685222, "longitude": -43.15898815139684, "uid": "42314380694", "password": "H6OHQq", "transaction": "true", "api": "login-authentication", "endpoint": "v1/login", "os": "Android 13", "phone_brand": "ZTE Axon 50 Ultra", "app_version": "2.0.3", "error": "0", "timestamp": "1730473039000.0"}
[2] - Mensagem enviada com sucesso! - {"imei": "833925317400834", "mac": "86:44:9B:0D:5C:56", "network": "TIM", "client_ip": "116.251.92.47", "latitude": -23.62298065695965, "longitude": -46.85060939121656, "uid": "33143910329", "password": "rA1BRv", "transaction": "true", "api": "login-authentication", "endpoint": "v1/login", "os": "Android 13 (Realme UI 4.0)", "phone_brand": "Realme 11", "app_version": "2.0.3", "error": "0", "timestamp": "1730426977000.0"}
[2] - Mensagem enviada com sucesso!

In [ ]:
import pandas as pd
import dash
from dash import dcc, html
import plotly.express as px
from dash.dependencies import Input, Output

from jupyter_dash import JupyterDash

data = [generate_random_data() for _ in range(100)]  

df = pd.DataFrame(data)

app = JupyterDash(__name__)

# Layout do app
app.layout = html.Div([
    html.H1("Mapa de Calor das Cidades"),
    dcc.Graph(
        id='heatmap',
        figure=px.scatter_mapbox(df, 
                                  lat="latitude", 
                                  lon="longitude", 
                                  color="latitude", 
                                  size_max=15,
                                  title="Mapa de Calor com Posições das Cidades",
                                  color_continuous_scale="Viridis")
            .update_layout(
                mapbox_style="carto-positron",  # Estilo básico do mapa
                mapbox_zoom=4,  # Nível de zoom do mapa
                mapbox_center={"lat": -5, "lon": -65}  # Centralizar o mapa
            )
    ),
])

# Rodar o servidor com modo inline
app.run_server(debug=True, mode='inline')